Exploratory analysis into the Zomato Bengaluru Data Set 

1. [Import Dependencies](#dependencies)
1. [View Data](#viewdata)
1. [Clean Up](#dropColumns)
1. [Diving into the Data](#diving)
    1. [Distribution of Ratings](#ratingDist)
    1. [Distribution of Meal Cost](#distOfMealCost)
    1. [Meal Cost vs. Rating](#ratingVsMealCost)
    1. [Is there a relationship between Rating and Votes?](#ratingVsVotes)
    1. [Top Ten Restaurant Types by Vote Count](#topTenByVote)
    1. [Top Ten Restaurant Types by Ratings](#topTenTypeByRate)
    1. [Top Ten Restaurants by Votes](#topTenRests)
    1. [Top Ten Restaurants by Rating](#topTenRatings)
    1. [Where are the Highest Rated Restaurants](#whereAreTheHighestRatedRestaurants)
    1. [Where are the Most Voted Restaurants](#cityTotalVotes)




**Import Dependencies**
<div id='dependencies'></div>

In [ ]:
%matplotlib inline
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns ; sns.set()
import os
# print(os.listdir("../input"))
import plotly
plotly.__version__
import plotly.plotly as py
import plotly.graph_objs as go

# **View Data**
<div id='viewdata'></div>

In [ ]:
rawData = pd.read_csv('../input/zomato.csv')
rawData.head()

# **Lets get a look at our data. **

In [ ]:
print('Data types')
print(rawData.dtypes)
print('Info')
print(rawData.info())
print('Rows and Columns')
print(rawData.shape)

It is evident that as the analysis futhers that we will need to change a few data types from object to int or floats. It is also seen that the rating column and dishes liked have a considerable aomount of null values that will be addresed. 

<div id='dropColumns'></div>

# **Clean the data up. **

1.    Drop unnecessary columns and rename columns. 
1.    No need for URL and Phone Numbers. 
1.    Clean up a restaurants name
1.    Drop null and new values from the rating 
1.    Remove ',' from cost_for_two column
 

In [ ]:
print(rawData.columns)
rawData.drop(['url','phone',],axis=1,inplace=True)
rawData = rawData.rename( columns={'approx_cost(for two people)':'cost_for_two','listed_in(type)':'restaurant_type','listed_in(city)':'city'} )
rawData = rawData.replace('SantÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂ© Spa Cuisine','Spa Cuisine')

In [ ]:
# Calculate percentage of restaurants with no rating. 
newRest = (rawData['rate'] == 'NEW').sum()
totalRatings= (rawData['rate'].count())
nanValues = (rawData['rate'].isnull()).sum()
percentOfNoRatings = ((nanValues + newRest)  / totalRatings * 100).round(2)
print (f'There are {totalRatings} restaurants in the Zomato data set that have ratings. Out of the total 51,717 restaurants {percentOfNoRatings}% are new or have no rating ')

For the rating column we will remove the /5 allowing us to analyze the data without errors. 
This will include removing NAN values and NEW restaurants. This is unfortunate with almost 23% of the restaurants having no ratings. 
    

In [ ]:
ratingCount = pd.DataFrame(rawData.groupby('rate')['rate'].count())
# drop na values
dataFiltered = rawData.dropna()
# filter out ratings that have a value of NEW
dataFiltered = dataFiltered[dataFiltered['rate'] != 'NEW']

# reduce rating column to just rating
dataFiltered['rate'] = dataFiltered['rate'].astype(str)
dataFiltered['rate'] = dataFiltered['rate'].apply(lambda rate: rate[:3])
dataFiltered['rate'] = dataFiltered['rate'].astype(float)

# remove ',' from cost for two column
dataFiltered['cost_for_two'] = dataFiltered['cost_for_two'].apply(lambda x: x.replace(',','')).astype(int)

<div id='diving'></div>
# Diving into the dataset. 


<div id='ratingDist'></div>
 **Distribution of Ratings**

In [ ]:
sns.set_style('white')
fig, ax = plt.subplots(figsize=(15,9))
sns.distplot(dataFiltered['rate'],bins=15,color="m",kde=False,)
plt.title('Distribution of Ratings')
plt.xlabel('Rating')
sns.despine()

<div id='distOfMealCost'></div>
**Distribution of Meal Cost**

In [ ]:
sns.set_style('white')
fig, ax = plt.subplots(figsize=(30,9))
sns.distplot(dataFiltered['cost_for_two'],color="m",kde=False,)
plt.title('Distribution of Meal Cost')
plt.xlabel('Cost for Two')
plt.xticks(dataFiltered['cost_for_two'].unique(),rotation='vertical')
plt.xlim(40,3000)
sns.despine()

<div id='ratingVsMealCost'></div>
**Rating vs. Meal Cost**

Lets see if there is any relationship between Ratings and the Meal cost for two people. 

In [ ]:
# Need to remove , from cost column // need to move this above meal cost dist
# dataFiltered['cost_for_two'] = dataFiltered['cost_for_two'].apply(lambda x: x.replace(',','')).astype(int)
# plotting
fig, ax = plt.subplots(figsize=(30,9))
ax = sns.scatterplot(y='rate',x='cost_for_two',data=dataFiltered, color='m',palette='plasma')
ax.set_title('Cost vs Ratings')
ax.set_ylabel('Ratings')
ax.set_xlabel('Cost for Two')
sns.set_style('white')
sns.despine()

We can see that once we pass around the 1,500 Rupee mark the ratings count lowers but does stay above 3.5, with most of the ratings being pretty well spread out for restaurants that are between 0 and 1,000 Ruppees. 

Something that struck me was if there was any correlation between how many cuisines were offered and ratings. 

   We see the highest votes go to restaurants that have between 3 and 4 cuisines, with a decline in votes for place that serve more than five. Pretty obivious, always better to focus a few than many. 

<div id='ratingVsVotes'></div>
**Ratings vs Votes**

In [ ]:
plt.figure(figsize=(20,5))

sns.regplot(x='rate',y='votes',data=dataFiltered,color='m')
plt.ylim(0,17500)
plt.xlabel('Rating')
plt.ylabel('Votes')
plt.title('Rating vs Votes')
sns.despine()

It's apparent that votes stay below the 2000 mark until a restaurants rating goes above 3.5.

In [ ]:
# create a column that has the number of cusines served
dataFiltered['cuisines_count'] = dataFiltered['cuisines'].apply(lambda x: len(x.split(','))).astype(int)
#plot it

plt.figure(figsize=(10,5))
sns.set_style('white')
sns.scatterplot(x='cuisines_count', y='votes',data=dataFiltered,palette='plasma')
sns.despine()
plt.ylabel('Votes')
plt.xlabel('Cuisines Served')

****Top Ten Restaurants Types by Vote Count
<div id='topTenByVote'></div>

In [ ]:
topTenRestTypesByVotes = pd.DataFrame(dataFiltered.groupby(['rest_type'])[['votes']].sum()).sort_values(by='votes',ascending=False).astype(int)
topTenRestTypesByVotes.reset_index(inplace=True)

plt.figure(figsize=(15,5))
sns.barplot(topTenRestTypesByVotes['votes'].iloc[:10],topTenRestTypesByVotes['rest_type'].iloc[:10],palette='plasma')
sns.despine()
plt.ylabel('Restaurant Type')
plt.xlabel('Total Votes')
plt.title('Top Ten Restaurant Types by Total Votes')

<div id='topTenTypeByRate'></div>

****Top Ten Restaurant Types by Ratings


In [ ]:
topTenTypeByRate = pd.DataFrame(dataFiltered.groupby(['rest_type','name'])['rate'].mean()).sort_values(by='rate',ascending=False).iloc[:10].reset_index()
topTenTypeByRate

plt.figure(figsize=(15,5))
sns.barplot(topTenTypeByRate['rate'].iloc[:10],topTenTypeByRate['rest_type'].iloc[:10],palette='plasma')
sns.despine()
plt.ylabel('Restaurant Type')
plt.xlabel('Average Rating')
plt.title('Top Ten Restaurant Types by Average Rating')

<div id='topTenRests'></div>
****Top Ten Restaurants by Total Votes


In [ ]:
topTenByVotes = pd.DataFrame(dataFiltered.groupby(['name','rest_type'])['votes'].sum()).sort_values(by='votes',ascending=False).iloc[0:10].reset_index()

plt.figure(figsize=(15,5))
sns.barplot(topTenByVotes['votes'].iloc[:10],topTenByVotes['name'].iloc[:10],palette='plasma')
sns.despine()
plt.ylabel('Restaurant')
plt.xlabel('Total Votes')
plt.title('Top Ten Restaurants by Total Votes')

<div id='topTenRatings'></div>
Top Ten Restaurants by Average Rating

In [ ]:
topTenByRatings = pd.DataFrame(dataFiltered.groupby(['name'])['rate'].mean()).sort_values(by='rate',ascending=False).reset_index()

fig, axs = plt.subplots(figsize=(10,5),)
sns.barplot(topTenByRatings['rate'].iloc[:10],topTenByRatings['name'].iloc[:10],palette='plasma')
sns.despine()
plt.ylabel('Restaurant')
plt.xlabel('Average Rating')
plt.title('Top Ten Restaurants by Average Rating')
plt.xlim(3.5,5.0)

<div id='whereAreTheHighestRatedRestaurants'></div>
**Where are the Highest Rated Restaurants?**

In [ ]:
highestRatedByLocation = pd.DataFrame(dataFiltered.groupby(['city'])['rate'].mean()).sort_values(by='rate',ascending=False).reset_index()
highestRatedByLocation

sns.set_style('white')

plt.figure(figsize=(5,5))
sns.barplot(y=highestRatedByLocation['city'].iloc[:10],x=highestRatedByLocation['rate'].iloc[:10],palette='plasma')
plt.title('Location of Highest Rated Restaurants')
plt.ylabel('City')
plt.xlabel('Average Rating')
plt.xlim(3.5,5.0)
sns.despine()


<div id='cityTotalVotes'></div>
**Where are the Most Voted Restaurants?**

In [ ]:
cityTotalVotes = pd.DataFrame(dataFiltered.groupby('city')['votes'].sum()).sort_values(by='votes',ascending=False).reset_index()
cityTotalVotes

sns.set_style('white')
plt.figure(figsize=(10,5))
sns.barplot(y=cityTotalVotes['city'].iloc[:10],x=cityTotalVotes['votes'].iloc[:10],palette='plasma')
plt.title('Location of Highest Rated Restaurants')
plt.ylabel('City')
plt.xlabel('Average Rating')
# plt.xlim(3.5,5.0)
sns.despine()

In [ ]:
# f, (ax1, ax2) = plt.subplots(2)
# sns.regplot(x, y, ax=ax1)
# sns.kdeplot(x, ax=ax2)

# f, (ax1, ax2) = plt.subplots(1,2,figsize=(20,5))
# sns.barplot(y=cityTotalVotes['city'].iloc[:10],x=cityTotalVotes['votes'].iloc[:10],palette='plasma', ax=ax1)

# plt.title('Location of Highest Rated Restaurants')
# plt.ylabel('City')
# plt.xlabel('Average Rating')
# plt.xlim(3.5,5.0)

# sns.barplot(y=highestRatedByLocation['city'].iloc[:10],x=highestRatedByLocation['rate'].iloc[:10],palette='plasma', ax=ax2)

# f[0] = plt.title('Location of Highest Rated Restaurants')
# ax2 = plt.ylabel('City')
# ax2 = plt.xlabel('Average Rating')

# f.tight_layout()